In [1]:
%pip install -q 'vanna[chromadb,ollama,postgres]'

  DEPRECATION: Building 'flasgger' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'flasgger'. Discussion can be found at https://github.com/pypa/pip/issues/6334
Note: you may need to restart the kernel to use updated packages.


In [1]:
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

In [2]:
class MyVanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={'model': 'llama3.2'})

In [3]:
vn.connect_to_sqlite('patients.db')

In [4]:
vn.train(ddl="CREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)")

Adding ddl: CREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)


'62e22abb-854a-580d-9ac9-f814093197ec-ddl'

In [5]:
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,62e22abb-854a-580d-9ac9-f814093197ec-ddl,None,"CREATE TABLE patients_details (Patient_ID, Age...",ddl


In [6]:
vn.ask(question="how many patients are there in database?", visualize=False, allow_llm_to_see_data=True)

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, ple

('SELECT COUNT(*) FROM patients_details',
    COUNT(*)
 0     10000,
 None)

In [10]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


WARNING	Thread(Thread-23 (process_request_thread)) chromadb.segment.impl.vector.local_persistent_hnsw:local_persistent_hnsw.py:query_vectors()- Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
WARNING	Thread(Thread-23 (process_request_thread)) chromadb.segment.impl.vector.local_persistent_hnsw:local_persistent_hnsw.py:query_vectors()- Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


In [ ]:
vn.